# Representing Time Periods

A notebook that demonstrates how to cast various time periods (weekly, monthly, quarterly) using *pandas*.

## Quarterly Time Series

Time series such as the NHS Digital Accident and Emergency figures are published as quarterly and monthly time series.

The following example shows how to cast the quarterly reports to a quarterly time period in *pandas*, with quarters aligned by financial year.

In [5]:
#Download example data file
url='https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2016/06/Quarterly-timeseries-May-2017.xls'
!wget -P data/ {url}

--2017-08-08 11:15:51--  https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2016/06/Quarterly-timeseries-May-2017.xls
Resolving www.england.nhs.uk... 54.230.199.109, 54.230.199.252, 54.230.199.172, ...
Connecting to www.england.nhs.uk|54.230.199.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92672 (90K) [application/vnd.ms-excel]
Saving to: 'data/Quarterly-timeseries-May-2017.xls'

Quarterly-timeserie 100%[=====================>]  90.50K  --.-KB/s   in 0.02s  

2017-08-08 11:15:52 (4.52 MB/s) - 'data/Quarterly-timeseries-May-2017.xls' saved [92672/92672]



In [6]:
import pandas as pd

#Preview data
dfw=pd.read_excel('data/Quarterly-timeseries-May-2017.xls',
                  skiprows=15,header=None,na_values='-').dropna(how='all').dropna(axis=1,how='all')
dfw.iloc[0,0]='Reporting'
dfw.iloc[1,0]='Code'

dfw= dfw.fillna(axis=1,method='ffill').T.set_index([0,1]).T.dropna(how='all').dropna(axis=1,how='all')
dfw=dfw.reset_index(drop=True)
dfw.head()

0 Reporting                                           A&E attendances  \
1      Code     Year           Quarter Type 1 Departments - Major A&E   
0       NaN  2004-05  Q1: April - June                        3377850   
1       NaN      NaN   Q2: July - Sept                        3381219   
2       NaN      NaN     Q3: Oct - Dec                        3257398   
3       NaN      NaN     Q4: Jan - Mar                        3249353   
4       NaN  2005-06  Q1: April - June                        3520931   

0                                        \
1 Type 2 Departments - Single Specialty   
0                                159966   
1                                156697   
2                                150643   
3                                151926   
4                                165801   

0                                                                     \
1 Type 3 Departments - Other A&E/Minor Injury Unit Total attendances   
0                                           964762           4502578   
1                                          1018779           4556695   
2                                           966886           4374927   
3                                          1001701           4402980   
4                                          1172846           4859578   

0 A&E attendances > 4 hours from arrival to admission, transfer or discharge  \
1                                             Type 1 Departments - Major A&E   
0                                             232110                           
1                                             182554                           
2                                             136953                           
3                                             115978                           
4                                              83845                           

0                                        \
1 Type 2 Departments - Single Specialty   
0                                  2612   
1                                  1830   
2                                  1415   
3                                   982   
4                                   742   

0                                                   \
1 Type 3 Departments - Other A&E/Minor Injury Unit   
0                                             3305   
1                                             2625   
2                                             1364   
3                                              687   
4                                              555   

0                                    ...                                     \
1                                    ...                                      
0                                    ...                                      
1                                    ...                                      
2                                    ...                                      
3                                    ...                                      
4                                    ...                                      

0                                                                             \
1 Percentage in 4 hours or less (type 1) Percentage in 4 hours or less (all)   
0                               0.931285                            0.947135   
1                               0.946009                             0.95896   
2                               0.957956                            0.968061   
3                               0.964307                             0.97328   
4                               0.976187                             0.98248   

0                Emergency Admissions                                      \
1 Emergency Admissions via Type 1 A&E Emergency Admissions via Type 2 A&E   
0                              651785                                6778   
1                              673452                                6670   
2               

Note how the time series is described in the data file across two columns: the financial year and the quarter:

In [8]:
dfw[[('Reporting','Year'),('Reporting','Quarter')]].head()

0 Reporting                  
1      Year           Quarter
0   2004-05  Q1: April - June
1       NaN   Q2: July - Sept
2       NaN     Q3: Oct - Dec
3       NaN     Q4: Jan - Mar
4   2005-06  Q1: April - June

Fill down on the financial year, and create a new *period* column joining those two elements.

In [9]:
dfw['Reporting','Year'].fillna(method='ffill',inplace=True)
dfw['Reporting','period']=dfw['Reporting','Year']+' '+dfw['Reporting','Quarter']
dfw[[('Reporting','Year'),('Reporting','Quarter'),('Reporting','period')]].head()

0 Reporting                                            
1      Year           Quarter                    period
0   2004-05  Q1: April - June  2004-05 Q1: April - June
1   2004-05   Q2: July - Sept   2004-05 Q2: July - Sept
2   2004-05     Q3: Oct - Dec     2004-05 Q3: Oct - Dec
3   2004-05     Q4: Jan - Mar     2004-05 Q4: Jan - Mar
4   2005-06  Q1: April - June  2005-06 Q1: April - June

Use the *pandas* ["anchored offsets"](http://pandas.pydata.org/pandas-docs/version/0.12.0/timeseries.html#anchored-offsets) to specifiy the quarterly period relative to the financial year.

In [16]:
#Return a date that we can use to anchor a period on
def getMonthYear(row):
    if not row['Reporting','Quarter'].startswith('Q'): return ''
    month=row['Reporting','Quarter'].split(':')[1].split('-')[0].strip()
    year=int(row['Reporting','Year'].split('-')[0])
    if month in ['Jan']:
        year= year+1
    #Following the conversion, the _quarter year specifies the calendar year in which the financial year ends
    #Return the date corresponding to the first day of the month in the calendar year the financial quarter applies to
    return pd.to_datetime("01-{}-{}".format(month[:3],year),format='%d-%b-%Y')

#Get a date corrsponding to the first day in the month of the first month in the quarter, by calendar year
#Use this date to align a particular quarterly period
#Q-MAR specifies a quarterly frequency, with year ending in March
dfw['Reporting','_quarter']=pd.PeriodIndex(dfw.apply(getMonthYear,axis=1), freq='Q-MAR')

#Sort the columns
dfw=dfw.sort_index(axis=1)
dfw=dfw.reindex(columns=['Reporting']+[c for c in dfw.columns.levels[0] if c!='Reporting'],level=0)

#Preview the data with the quarter time period column defined
dfw[[('Reporting','Year'),('Reporting','Quarter'),('Reporting','period'),('Reporting','_quarter')]].head()

0 Reporting                                                     
1      Year           Quarter                    period _quarter
0   2004-05  Q1: April - June  2004-05 Q1: April - June   2005Q1
1   2004-05   Q2: July - Sept   2004-05 Q2: July - Sept   2005Q2
2   2004-05     Q3: Oct - Dec     2004-05 Q3: Oct - Dec   2005Q3
3   2004-05     Q4: Jan - Mar     2004-05 Q4: Jan - Mar   2005Q4
4   2005-06  Q1: April - June  2005-06 Q1: April - June   2006Q1

## Monthly Time Series

NHS Digital Accident and Emergency figures are also published as a monthly time series.

In [18]:
url='https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2017/06/May-17-Monthly-AE-Timeseries-with-growth-charts.xlsx'
!wget -P data/ {url}

--2017-08-08 11:31:25--  https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2017/06/May-17-Monthly-AE-Timeseries-with-growth-charts.xlsx
Resolving www.england.nhs.uk... 54.230.199.30, 54.230.199.113, 54.230.199.252, ...
Connecting to www.england.nhs.uk|54.230.199.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111779 (109K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: 'data/May-17-Monthly-AE-Timeseries-with-growth-charts.xlsx'

May-17-Monthly-AE-T 100%[=====================>] 109.16K  --.-KB/s   in 0.02s  

2017-08-08 11:31:26 (4.53 MB/s) - 'data/May-17-Monthly-AE-Timeseries-with-growth-charts.xlsx' saved [111779/111779]



In [20]:
dfw=pd.read_excel('data/May-17-Monthly-AE-Timeseries-with-growth-charts.xlsx',skiprows=16,header=None,na_values='-').dropna(how='all').dropna(axis=1,how='all')
dfw.iloc[0,0]='Reporting'
dfw.iloc[1,0]='Code'

dfw= dfw.fillna(axis=1,method='ffill').T.set_index([0,1]).T.dropna(how='all').dropna(axis=1,how='all')
dfw.head()

0 Reporting                                     A&E attendances  \
1      Code               Period Type 1 Departments - Major A&E   
2       NaN  2010-08-01 00:00:00                        1138652   
3       NaN  2010-09-01 00:00:00                        1150728   
4       NaN  2010-10-01 00:00:00                        1163143   
5       NaN  2010-11-01 00:00:00                    1.11129e+06   
6       NaN  2010-12-01 00:00:00                     1.1592e+06   

0                                        \
1 Type 2 Departments - Single Specialty   
2                                 54371   
3                                 55181   
4                                 54961   
5                               53727.4   
6                               45536.4   

0                                                                     \
1 Type 3 Departments - Other A&E/Minor Injury Unit Total Attendances   
2                                           559358           1752381   
3                                           550359           1756268   
4                                           583244           1801348   
5                                           486005       1.65103e+06   
6                                           533001       1.73774e+06   

0 A&E attendances > 4 hours from arrival to admission, transfer or discharge  \
1                                             Type 1 Departments - Major A&E   
2                                              32664                           
3                                              40416                           
4                                              46467                           
5                                            45838.4                           
6                                            88475.3                           

0                                        \
1 Type 2 Departments - Single Specialty   
2                                    97   
3                                   142   
4                                   141   
5                               143.429   
6                               140.571   

0                                                   \
1 Type 3 Departments - Other A&E/Minor Injury Unit   
2                                              423   
3                                              593   
4                                              806   
5                                          454.571   
6                                          1301.43   

0                                            ...                  \
1 Total Attendances > 4 hours                ...                   
2                       33184                ...                   
3                       41151                ...                   
4                       47414                ...                   
5                     46436.4                ...                   
6                     89917.3                ...                   

0                                                    Emergency Admissions  \
1 Percentage in 4 hours or less (all) Emergency Admissions via Type 1 A&E   
2                            0.981063                              287438   
3                            0.976569                              293991   
4                            0.973679                              303452   
5                            0.971874                              297832   
6                            0.948256                              318602   

0                                      \
1 Emergency Admissions via Type 2 A&E   
2                                5367   
3                                5543   
4                                5485   
5                             5731.14   
6                                6277   

0                                            \
1 Emergency Admissions via Type 3 and 4 A&E   
2                                      8081   
3                                  

The *pandas* `dt.to_period('M')` function returns a monthly period corresponding to the month within which a specified date falls.

In [27]:
#Cast the date corresponding to the month start as a month period starting on that date
dfw['Reporting','_period']=pd.to_datetime(dfw['Reporting','Period'],
                                          format='%d/%m/%Y').dt.to_period('M')

dfw=dfw.sort_index(axis=1)
dfw=dfw.reindex(columns=['Reporting']+[c for c in dfw.columns.levels[0] if c!='Reporting'],level=0)
dfw[[('Reporting','Period'),('Reporting','_period')]].head()

0            Reporting        
1               Period _period
2  2010-08-01 00:00:00 2010-08
3  2010-09-01 00:00:00 2010-09
4  2010-10-01 00:00:00 2010-10
5  2010-11-01 00:00:00 2010-11
6  2010-12-01 00:00:00 2010-12

## Weekly Time Series

The NHS used to publish weekly time series for A & E figures. Archival data is [still available](https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/statistical-work-areasae-waiting-times-and-activityweekly-ae-sitreps-2015-16/).

In [28]:
url='https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2015/04/2015.06.28-AE-TimeseriesBaG87.xls'
!wget -P data/ {url}

--2017-08-08 11:40:14--  https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2015/04/2015.06.28-AE-TimeseriesBaG87.xls
Resolving www.england.nhs.uk... 54.230.199.90, 54.230.199.191, 54.230.199.113, ...
Connecting to www.england.nhs.uk|54.230.199.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84480 (82K) [application/vnd.ms-excel]
Saving to: 'data/2015.06.28-AE-TimeseriesBaG87.xls'

2015.06.28-AE-Times 100%[=====================>]  82.50K  --.-KB/s   in 0.03s  

2017-08-08 11:40:15 (3.18 MB/s) - 'data/2015.06.28-AE-TimeseriesBaG87.xls' saved [84480/84480]



In [33]:
dfw=pd.read_excel('data/2015.06.28-AE-TimeseriesBaG87.xls',
                  skiprows=14,header=None,na_values='-').dropna(how='all').dropna(axis=1,how='all')
dfw.iloc[0,0]='Reporting'
dfw.iloc[1,0]='Code'

dfw= dfw.fillna(axis=1,method='ffill').T.set_index([0,1]).T.dropna(how='all').dropna(axis=1,how='all')

dfw.head()

0 Reporting                                      A&E attendances  \
1      Code                Period Type 1 Departments - Major A&E   
2       NaN  2015-16 Year to Date                        3702458   
4       Eng        W/E 07/11/2010                         267142   
5       Eng        W/E 14/11/2010                         256893   
6       Eng        W/E 21/11/2010                         260958   
7       Eng        W/E 28/11/2010                         256061   

0                                        \
1 Type 2 Departments - Single Specialty   
2                                159697   
4                                 12806   
5                                 13135   
6                                 12751   
7                                 12126   

0                                                                     \
1 Type 3 Departments - Other A&E/Minor Injury Unit Total Attendances   
2                                          1850023           5712178   
4                                           111547            391495   
5                                           115657            385685   
6                                           115034            388743   
7                                           113086            381273   

0 A&E attendances > 4 hours from arrival to admission, transfer or discharge  \
1                                             Type 1 Departments - Major A&E   
2                                             329060                           
4                                              11827                           
5                                              10223                           
6                                               9790                           
7                                              10087                           

0                                        \
1 Type 2 Departments - Single Specialty   
2                                  1278   
4                                    40   
5                                    29   
6                                    33   
7                                    36   

0                                                   \
1 Type 3 Departments - Other A&E/Minor Injury Unit   
2                                             6765   
4                                               85   
5                                              131   
6                                               94   
7                                               96   

0                                                                     \
1 Total Attendances > 4 hours Percentage in 4 hours or less (type 1)   
2                      337103                               0.911124   
4                       11952                               0.955728   
5                       10383                               0.960205   
6                        9917                               0.962484   
7                       10219                               0.960607   

0                                                    Emergency Admissions  \
1 Percentage in 4 hours or less (all) Emergency Admissions via Type 1 A&E   
2                            0.940985                              994256   
4                            0.969471                               73589   
5                            0.973079                               67526   
6                             0.97449                               68990   
7                            0.973198                               68181   

0                                      \
1 Emergency Admissions via Type 2 A&E   
2                                4077   
4                                1372   
5                                1340   
6                                1343   
7                                1305   

0                                            \
1 Emergency Admissions via Type 3 and 4 A&E   
2                                     12543   


The *pandas* `.to_period('W')` function returns a week period, starting on a Monday, that a given date falls within.

In [34]:
#Filter to retain rows that have a W/E period specified
dfw=dfw[dfw[('Reporting','Period')].str.startswith('W/E')]

#Remove the W/E prefix and use the date to define
dfw['Reporting','_period']=pd.to_datetime(dfw['Reporting','Period'].str.replace('W/E ',''),
                                          format='%d/%m/%Y').dt.to_period('W') 

dfw[[('Reporting','Period'),('Reporting','_period')]].head()

0       Reporting                      
1          Period               _period
4  W/E 07/11/2010 2010-11-01/2010-11-07
5  W/E 14/11/2010 2010-11-08/2010-11-14
6  W/E 21/11/2010 2010-11-15/2010-11-21
7  W/E 28/11/2010 2010-11-22/2010-11-28
8  W/E 05/12/2010 2010-11-29/2010-12-05